# This script reads metadata in a .csv, cleans and formats the data, and exports the result as a .csv for use in climate research.

In [2]:
import pandas as pd

In [13]:
# This is the raw metadata generated by the csvgen tool
df = pd.read_csv('https://climatedata.oscer.ou.edu/infx/climatedata_updated.csv ')

Index(['variable', 'institute', 'model', 'domain', 'product', 'project',
       'realm', 'ensemble', 'experiment', 'time', 'time_frequency', 'filename',
       'local_file', 'cmor_table', 'data_node', 'size', 'version', 'url',
       'latest', 'checksum', 'checksum_type'],
      dtype='object')

In [15]:
# This is the table of variable names compiled from CORDEX and the IPCC standard output
df_lookup = pd.read_csv('deduped_lookup.csv')

In [16]:
# This is the table that contains dimension labels for each variable
df_dimensions = pd.read_csv('deduped_dimensions.csv')

In [17]:
# This merges the raw metadata and variable name table using the 'variable' column as the join index
complete_variables = pd.merge(df,df_lookup, on='variable')

In [18]:
df = pd.merge(complete_variables, df_dimensions, on='variable')

## Update Path for local and OSCER path


In [19]:
# Formats fullpath to export path to data in OSCER

def newpath(item):
    root = item.split("/")[1]
    remaining_path = "/".join(item.split("/")[2:])
    if root == "data":
        fullpath = f"/condo/climatedata3/{remaining_path}"
    else:
        fullpath = f"/condo/climate{root}/{remaining_path}"
    return fullpath


df['OSCER_schooner_path'] = df['local_file'].apply(newpath)

In [20]:
# Creates a copy of 'local_file' paths as 'SCCASC_climatedata_path'

df['SCCASC_climatedata_path'] = df['local_file']

## Find unique entries

In [26]:
unique = df[~df.duplicated(['size', 'checksum'], False)]

Index(['variable', 'institute', 'model', 'domain', 'product', 'project',
       'realm', 'ensemble', 'experiment', 'time', 'time_frequency', 'filename',
       'local_file', 'cmor_table', 'data_node', 'size', 'version', 'url',
       'latest', 'checksum', 'checksum_type', 'variable_standard_name',
       'variable_long_name', 'dimensions', 'OSCER_schooner_path',
       'SCCASC_climatedata_path'],
      dtype='object')

## Deduplicate entries and recombine

In [42]:
df.groupby(['time', 'filename', 'size','checksum'], as_index=False)['version'].max()

,time,filename,size,checksum,version
0,185001-185012,hus_Amon_EC-EARTH_historical_r1i1p1_185001-185...,39342004,747cb3039a2f997e097683c5e8b84d05eef78806dac321...,v20131231
1,185001-185012,pr_Amon_EC-EARTH_historical_r6i1p1_185001-1850...,2477816,d176225080b45b3319ca1fec6887b732aa3c9abe9afecf...,v20130315
2,185001-185012,prc_Amon_EC-EARTH_historical_r6i1p1_185001-185...,2477772,50b9c7387384acde93a3c49c9f5a2fd5ae10e101675f73...,v20130315
3,185001-185012,ta_Amon_EC-EARTH_historical_r1i1p1_185001-1850...,39341996,f4935ca011269d6256743fedffca86511ef69f69cdd7d6...,v20131231
4,185001-185012,ua_Amon_EC-EARTH_historical_r1i1p1_185001-1850...,39341904,cb2be657a1122117dcbfb7a85554763d7f61b29485ec07...,r1i1p1
...,...,...,...,...,...
188586,21001201-21001230,tos_day_HadGEM2-ES_rcp60_r3i1p1_21001201-21001...,9354572,6ef6e172ada02db5a69a8c9971913f3442895c98bef6ca...,v20111209
188587,21001201-21001230,tos_day_HadGEM2-ES_rcp60_r4i1p1_21001201-21001...,9354572,5e315cca91391f01540ce84e9dd7c1b5760968775f1d1d...,v20111209
188588,21001201-21001230,tos_day_HadGEM2-ES_rcp85_r2i1p1_21001201-21001...,9353916,2c93f0c6cb9be667ac03b887033dc0dd7a545777272865...,v20120114
188589,21001201-21001230,tos_day_HadGEM2-ES_rcp85_r3i1p1_21001201-21001...,9353916,cc345f7c376914d593d1933a45d0a5166cf7e27cfef8e7...,v20120114


In [41]:
deduplicated

,time,filename,size,checksum,version
0,185001-185012,hus_Amon_EC-EARTH_historical_r1i1p1_185001-185...,39342004,747cb3039a2f997e097683c5e8b84d05eef78806dac321...,v20131231
1,185001-185012,pr_Amon_EC-EARTH_historical_r6i1p1_185001-1850...,2477816,d176225080b45b3319ca1fec6887b732aa3c9abe9afecf...,v20130315
2,185001-185012,prc_Amon_EC-EARTH_historical_r6i1p1_185001-185...,2477772,50b9c7387384acde93a3c49c9f5a2fd5ae10e101675f73...,v20130315
3,185001-185012,ta_Amon_EC-EARTH_historical_r1i1p1_185001-1850...,39341996,f4935ca011269d6256743fedffca86511ef69f69cdd7d6...,v20131231
4,185001-185012,ua_Amon_EC-EARTH_historical_r1i1p1_185001-1850...,39341904,cb2be657a1122117dcbfb7a85554763d7f61b29485ec07...,r1i1p1
...,...,...,...,...,...
188586,21001201-21001230,tos_day_HadGEM2-ES_rcp60_r3i1p1_21001201-21001...,9354572,6ef6e172ada02db5a69a8c9971913f3442895c98bef6ca...,v20111209
188587,21001201-21001230,tos_day_HadGEM2-ES_rcp60_r4i1p1_21001201-21001...,9354572,5e315cca91391f01540ce84e9dd7c1b5760968775f1d1d...,v20111209
188588,21001201-21001230,tos_day_HadGEM2-ES_rcp85_r2i1p1_21001201-21001...,9353916,2c93f0c6cb9be667ac03b887033dc0dd7a545777272865...,v20120114
188589,21001201-21001230,tos_day_HadGEM2-ES_rcp85_r3i1p1_21001201-21001...,9353916,cc345f7c376914d593d1933a45d0a5166cf7e27cfef8e7...,v20120114


In [32]:
complete_deduplicated = pd.concat([unique, deduplicated])

In [33]:
# Extract beginning year
complete_deduplicated['beg_year'] = complete_deduplicated.time.str.split("-").apply(lambda x: int(x[0][0:4]))

In [34]:
# Exclude anything starting after year 2101
cleaned = complete_deduplicated[complete_deduplicated.beg_year < 2101]

In [35]:
# Define column headers and sort data
headers = (['variable', 'variable_standard_name', 'variable_long_name', 'institute',
       'model', 'domain', 'dimensions', 'project', 'realm', 'ensemble',
       'experiment', 'time_frequency', 'time', 'version', 'filename', 'size',
       'OSCER_schooner_path', 'SCCASC_climatedata_path', 'checksum_type',
       'checksum'])
cleaned = cleaned.sort_values(['variable', 'model', 'experiment', 'time_frequency'])

In [36]:
# Reorder columns
df = cleaned[['variable', 'variable_standard_name', 'variable_long_name', 'institute',
       'model', 'domain', 'dimensions', 'project', 'realm', 'ensemble',
       'experiment', 'time_frequency', 'time', 'version', 'filename', 'size',
       'OSCER_schooner_path', 'SCCASC_climatedata_path', 'checksum_type',
       'checksum']]

In [37]:
df

,variable,variable_standard_name,variable_long_name,institute,model,domain,dimensions,project,realm,ensemble,experiment,time_frequency,time,version,filename,size,OSCER_schooner_path,SCCASC_climatedata_path,checksum_type,checksum
0,clt,cloud_area_fraction,Total Cloud Fraction,CSIRO-BOM,ACCESS1-0,day,2D,CMIP5,atmos,r1i1p1,historical,day,20000101-20051231,v20131108,clt_day_ACCESS1-0_historical_r1i1p1_20000101-2...,244170468,/condo/climatedata4/synda/data/cmip5/output1/C...,/data4/synda/data/cmip5/output1/CSIRO-BOM/ACCE...,sha256,8e98462e34c0d6bb52a1c66e746fb508be3a5764819d21...
1,clt,cloud_area_fraction,Total Cloud Fraction,CSIRO-BOM,ACCESS1-0,day,2D,CMIP5,atmos,r1i1p1,historical,day,19500101-19741231,v20131108,clt_day_ACCESS1-0_historical_r1i1p1_19500101-1...,1017064044,/condo/climatedata4/synda/data/cmip5/output1/C...,/data4/synda/data/cmip5/output1/CSIRO-BOM/ACCE...,sha256,653827333b43585cf765d88452bf7bc50b20d9892e506c...
2,clt,cloud_area_fraction,Total Cloud Fraction,CSIRO-BOM,ACCESS1-0,day,2D,CMIP5,atmos,r1i1p1,historical,day,19750101-19991231,v20131108,clt_day_ACCESS1-0_historical_r1i1p1_19750101-1...,1017064044,/condo/climatedata4/synda/data/cmip5/output1/C...,/data4/synda/data/cmip5/output1/CSIRO-BOM/ACCE...,sha256,bd0010424edbdbc6a6db509db3eefeb55c92ba15afab2e...
3,clt,cloud_area_fraction,Total Cloud Fraction,CSIRO-BOM,ACCESS1-0,day,2D,CMIP5,atmos,r1i1p1,historical,day,20000101-20051231,v4,clt_day_ACCESS1-0_historical_r1i1p1_20000101-2...,244170468,/condo/climatedata4/synda/data/cmip5/output1/C...,/data4/synda/data/cmip5/output1/CSIRO-BOM/ACCE...,sha256,72c18e9934cd861a60d0d2d5fec5b4d2b28e8bd3840936...
4,clt,cloud_area_fraction,Total Cloud Fraction,CSIRO-BOM,ACCESS1-0,day,2D,CMIP5,atmos,r1i1p1,historical,day,19500101-19741231,v4,clt_day_ACCESS1-0_historical_r1i1p1_19500101-1...,1017064044,/condo/climatedata4/synda/data/cmip5/output1/C...,/data4/synda/data/cmip5/output1/CSIRO-BOM/ACCE...,sha256,ca069dd0089d86d345ec01ac99d810b1d5800710959b72...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147319,zg,geopotential_height,Geopotential Height,INM,inmcm4,Amon,3D,CMIP5,atmos,r1i1p1,rcp85,mon,205601-206512,v20130207,zg_Amon_inmcm4_rcp85_r1i1p1_205601-206512.nc,176274224,/condo/climatedata1/synda/data/cmip5/output1/I...,/data1/synda/data/cmip5/output1/INM/inmcm4/rcp...,sha256,34cb47c040232fc7d0b4d30e7409c66a6b565fe5e195a4...
156006,zg,geopotential_height,Geopotential Height,INM,inmcm4,Amon,3D,CMIP5,atmos,r1i1p1,rcp85,mon,206601-207512,v20130207,zg_Amon_inmcm4_rcp85_r1i1p1_206601-207512.nc,176274224,/condo/climatedata1/synda/data/cmip5/output1/I...,/data1/synda/data/cmip5/output1/INM/inmcm4/rcp...,sha256,7df9197add1678054b8464b02b97417c8d83e6ba5c3be0...
164891,zg,geopotential_height,Geopotential Height,INM,inmcm4,Amon,3D,CMIP5,atmos,r1i1p1,rcp85,mon,207601-208512,v20130207,zg_Amon_inmcm4_rcp85_r1i1p1_207601-208512.nc,176274224,/condo/climatedata1/synda/data/cmip5/output1/I...,/data1/synda/data/cmip5/output1/INM/inmcm4/rcp...,sha256,ab8a381771d0d2df6e790540d222c61d57378c23a01d0d...
175434,zg,geopotential_height,Geopotential Height,INM,inmcm4,Amon,3D,CMIP5,atmos,r1i1p1,rcp85,mon,208601-209512,v20130207,zg_Amon_inmcm4_rcp85_r1i1p1_208601-209512.nc,176274224,/condo/climatedata1/synda/data/cmip5/output1/I...,/data1/synda/data/cmip5/output1/INM/inmcm4/rcp...,sha256,8db544611a8ca47c1e035b29df8d69208868a78e1a2a05...


In [38]:
# Export cleaned, deduplicated .csv
cleaned.to_csv("CMIP5_climatedata_full.csv", index=False, columns=headers)